In [94]:
import argparse
import logging
import sys

import numpy as np
import heapq
from twnews import defaults
from twnews.dataset.dataset import Dataset
from twnews.resolver import resolve, url_analyse
from twnews.utils.memoize import memo_process, load, dump
from twnews.utils.text_processors import lemmatize_texts, build_tf_idf_matrix
from twnews.wtmf.wtmf import WTMF
from scipy.sparse import find, csr_matrix, csc_matrix

In [144]:
news = load('news_applied')[:10]
tweets = load('documents_applied')[:10]

print len(news), len(tweets)
#recommendation = recommend(news, tweets, top_size=10)

10 10


In [152]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
def convert_to_compare_matrix(documents):
    dim = documents[0].compare_vector.shape[0]

    data, row_idxs, column_idxs = [], [], []
    for column_idx, document in enumerate(documents):
        rows,_,x=  find(document.compare_vector)
        for i, value in enumerate(x):
            data.append(x[i])
            row_idxs.append(rows[i])
            column_idxs.append(column_idx)
            
    compare_matrix = sparse.csr_matrix((data, (row_idxs, column_idxs)), shape=(dim, len(documents)))
    return compare_matrix

tweets_matrix = convert_to_compare_matrix(tweets[:10])
print 'tweets converted'
news_matrix = convert_to_compare_matrix(news)
print 'news converted'



tweets converted
news converted


In [153]:
mat = cosine_similarity(tweets_matrix.T, news_matrix.T)


In [156]:
print mat

[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.01340223  0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.02844482  0.          0.          0.          0.          0.
   0.02469734  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.01474232  0.          0.02969134
   0.03726059  0.02312094  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.04299861]
 [ 0.02783191  0.          0.          0.05294611  0.   

In [155]:
from scipy.sparse import find
def generate(data, corpus):
    v = data.compare_vector
    idx,_,_ = find(v)
    k = [corpus[x] for x in idx]
    for x in k: print x, 
    print ''
    

In [157]:
recoms = load('recommendation')


In [158]:
with open('/home/avybornov/tfidf_result','w') as f:
    for tweet, news in recoms[:100]:
        f.write('TWEET: {TWEET}\n{URLS}\n'.format(
            TWEET=tweet,
            URLS=tweet.urls))
        
        #f.write(str(tweet.urls) + '')
        for idx, news_tuple in enumerate(news):
            _news, score = news_tuple
            target = 'TARGET' if _news.link in tweet.urls else ''
            f.write('\t{ID}) {TARGET}NEWS: {NEWS} {SCORE} {LINK}\n'.format(
                ID=idx+1,
                TARGET=target,
                NEWS=_news,
                SCORE=score,
                LINK=_news.link))
    

In [105]:
RR = 0.0
for tweet, news in recoms:
    for i, single_news_tuple in enumerate(news):
        single_news, score = single_news_tuple
        if single_news.link in tweet.urls:
            RR += 1.0/(i+1)
            break
print RR/len(recoms)

0.786178571429


In [118]:
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    _, v1_idxs, value = sparse.find(v1)
    _, v2_idxs, value = sparse.find(v2)
    
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in set(np.append(v1_idxs, v2_idxs)):
        x = v1[(0,i)]; y = v2[(0,i)]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)


In [121]:
from scipy import sparse
x = sparse.csr_matrix([0,0,1,2])
y = sparse.csr_matrix([1,0,1,1])

In [122]:
print cosine_similarity(x,y)

0
2
3
0.774596669241


In [75]:
for x in a:
    print x
    break

  (0, 2)	1
  (0, 3)	2


In [78]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity 
a = cosine_similarity(x,y)
#sklearn.metrics.pairwise.pairwise_distances(x,y,metric='cosine')

In [83]:
print a[0][0]

0.774596669241


In [113]:
_, v1_idxs, value = sparse.find(x)
v1_idxs

array([2, 3], dtype=int32)

In [117]:
set(np.append(v1_idxs, v1_idxs))

{2, 3}

In [135]:
corpus, tf_idf_matrix = load('tf_idf_corpus')
print tf_idf_matrix.shape
print tf_idf_matrix[:,1].shape
tf_idf_matrix = sparse.csr_matrix(tf_idf_matrix)
print tf_idf_matrix.shape
print tf_idf_matrix[:,1].shape

(25290, 18035)
(25290, 1)
(25290, 18035)
(25290, 1)


In [137]:
P, Q = load('model_1_3')
print Q.shape
print Q[:,1].shape
print find(Q[:,1])


(3, 18035)
(3, 1)


NameError: name 'find' is not defined

In [130]:
tf_idf_matrix

<25290x18035 sparse matrix of type '<type 'numpy.float64'>'
	with 314036 stored elements in Compressed Sparse Column format>